## <font color='157699'> Librerías

In [2]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("default")

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression,  f_classif, mutual_info_classif, chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegressionCV, Ridge

import pandas as pd
import a_funciones as funciones
import sqlite3 as sql

## <font color='157699'> Importación de datos

In [3]:
# Crear conexíon 
conn = sql.connect('data/my_database.db')
#conn = sql.connect('C:\\Users\\ESTEBAN\\Desktop\\Proyecto_RRHH\\data\\db')
curr = conn.cursor()
# Leer datos para 2016 
df_2015 = pd.read_sql("SELECT * FROM processed_data_2015_modificado", conn)

In [4]:
df_2015.head()

,EmployeeID,Age,BusinessTravel,Department,DistanceFromHome,Education,JobRole,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,JobSatisfaction,WorkLifeBalance,renuncia2016
0,1,51,Travel_Rarely,Sales,6,2,Healthcare Representative,131160,1.0,0.11,6,1,0,4.0,2.0,0
1,2,31,Travel_Frequently,Research & Development,10,1,Research Scientist,41890,0.0,0.23,3,5,1,2.0,4.0,1
2,3,32,Travel_Frequently,Research & Development,17,4,Sales Executive,193280,1.0,0.15,2,5,0,2.0,1.0,0
3,4,38,Non-Travel,Research & Development,2,5,Human Resources,83210,3.0,0.11,5,8,7,4.0,3.0,0
4,5,32,Travel_Rarely,Research & Development,10,1,Sales Executive,23420,4.0,0.12,2,6,0,1.0,3.0,0


### <font color='157699'> Dumizar variables categóricas

In [5]:
df_2015['JobSatisfaction'] = df_2015['JobSatisfaction'].astype(str)
df_2015['WorkLifeBalance'] = df_2015['WorkLifeBalance'].astype(str)
df_2015['NumCompaniesWorked'] = df_2015['NumCompaniesWorked'].astype(int)
df_2015['Education'] = df_2015['Education'].astype(str)

In [6]:
columnas_dumizar = ['BusinessTravel', 'Department', 'JobRole',
                    'JobSatisfaction', 'WorkLifeBalance', 'Education']

df_dummy = pd.get_dummies(df_2015[columnas_dumizar])
df_dummy.head()

,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,JobRole_Healthcare Representative,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,...,JobSatisfaction_4.0,WorkLifeBalance_1.0,WorkLifeBalance_2.0,WorkLifeBalance_3.0,WorkLifeBalance_4.0,Education_1,Education_2,Education_3,Education_4,Education_5
0,False,False,True,False,False,True,True,False,False,False,...,True,False,True,False,False,False,True,False,False,False
1,False,True,False,False,True,False,False,False,False,False,...,False,False,False,False,True,True,False,False,False,False
2,False,True,False,False,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
3,True,False,False,False,True,False,False,True,False,False,...,True,False,False,True,False,False,False,False,False,True
4,False,False,True,False,True,False,False,False,False,False,...,False,False,False,True,False,True,False,False,False,False


In [7]:
y = df_2015['renuncia2016']

In [8]:
x_numeric = df_2015.select_dtypes(int)
del x_numeric['renuncia2016']

In [9]:
df_normalizada = funciones.escalar_datos(x_numeric)

In [10]:
df_final = pd.concat([df_normalizada, df_dummy], axis=1)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4308 entries, 0 to 4307
Data columns (total 35 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Age                                4308 non-null   float64
 1   DistanceFromHome                   4308 non-null   float64
 2   MonthlyIncome                      4308 non-null   float64
 3   NumCompaniesWorked                 4308 non-null   float64
 4   TrainingTimesLastYear              4308 non-null   float64
 5   YearsAtCompany                     4308 non-null   float64
 6   YearsSinceLastPromotion            4308 non-null   float64
 7   BusinessTravel_Non-Travel          4308 non-null   bool   
 8   BusinessTravel_Travel_Frequently   4308 non-null   bool   
 9   BusinessTravel_Travel_Rarely       4308 non-null   bool   
 10  Department_Human Resources         4308 non-null   bool   
 11  Department_Research & Development  4308 non-null   bool 

## <font color='056938'>Métodos Wrapper 

### <font color='157699'> RFE (Recursive Feature Elimination)

In [11]:

#  Estimador en este caso para regresión logística (problema de clasificación binaria)
model = LogisticRegressionCV()

# Obtener columnas seleciconadas
X_new = funciones.recursive_feature_selection(df_final, y, model, 28)

# Nuevo conjunto de datos
df_new = df_final.iloc[:,X_new]
df_new.head()

Num Features: 28
Selected Features: [ True False  True  True  True  True  True  True  True  True  True  True
  True  True False  True  True  True  True  True  True  True  True False
 False  True  True  True  True  True False  True False False  True]
Feature Ranking: [1 8 1 1 1 1 1 1 1 1 1 1 1 1 6 1 1 1 1 1 1 1 1 5 4 1 1 1 1 1 7 1 2 3 1]


,Age,MonthlyIncome,NumCompaniesWorked,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,...,JobRole_Sales Executive,JobRole_Sales Representative,JobSatisfaction_1.0,JobSatisfaction_4.0,WorkLifeBalance_1.0,WorkLifeBalance_2.0,WorkLifeBalance_3.0,WorkLifeBalance_4.0,Education_2,Education_5
0,1.539396,1.405220,-0.677827,2.483743,-0.980679,-0.677540,False,False,True,False,...,False,False,False,True,False,True,False,False,True,False
1,-0.648748,-0.492641,-1.078386,0.157664,-0.329864,-0.368046,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
2,-0.539341,2.725878,-0.677827,-0.617696,-0.329864,-0.677540,False,True,False,False,...,True,False,False,False,True,False,False,False,False,False
3,0.117102,0.385813,0.123292,1.708383,0.158247,1.488922,True,False,False,False,...,False,False,False,True,False,False,True,False,False,True
4,-0.539341,-0.885310,0.523851,-0.617696,-0.167160,-0.677540,False,False,True,False,...,True,False,True,False,False,False,True,False,False,False


In [12]:
df_new = pd.concat([df_2015['EmployeeID'],df_final, df_2015['renuncia2016']], axis=1)
df_new.columns

Index(['EmployeeID', 'Age', 'DistanceFromHome', 'MonthlyIncome',
       'NumCompaniesWorked', 'TrainingTimesLastYear', 'YearsAtCompany',
       'YearsSinceLastPromotion', 'BusinessTravel_Non-Travel',
       'BusinessTravel_Travel_Frequently', 'BusinessTravel_Travel_Rarely',
       'Department_Human Resources', 'Department_Research & Development',
       'Department_Sales', 'JobRole_Healthcare Representative',
       'JobRole_Human Resources', 'JobRole_Laboratory Technician',
       'JobRole_Manager', 'JobRole_Manufacturing Director',
       'JobRole_Research Director', 'JobRole_Research Scientist',
       'JobRole_Sales Executive', 'JobRole_Sales Representative',
       'JobSatisfaction_1.0', 'JobSatisfaction_2.0', 'JobSatisfaction_3.0',
       'JobSatisfaction_4.0', 'WorkLifeBalance_1.0', 'WorkLifeBalance_2.0',
       'WorkLifeBalance_3.0', 'WorkLifeBalance_4.0', 'Education_1',
       'Education_2', 'Education_3', 'Education_4', 'Education_5',
       'renuncia2016'],
      dtype='obje

In [13]:
#Subir el DataFrame modificado a una nueva tabla en la base de datos SQLite
df_new.to_sql('v_seleccionadas', conn, if_exists='replace', index=False)

# Cerrar la conexión
conn.close()